### **IMPORTACIONES**

In [45]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

### Variable de entorno

In [46]:
load_dotenv() 

api_key = os.getenv("GOOGLE_API_KEY")

### Subir PDF (PyPDFLoader)

In [47]:
def upload_pdf(file_path):
    try: 
        loader = PyPDFLoader(file_path)
        loader = loader.lazy_load()
        
        text = ""
        for page in loader:
            text += page.page_content + "\n"
        return text
    except Exception as e:
        print(f"Error al cargar el PDF: {e}")
        return []

### Separar contenido del PDF (CharacterTextSplitter)

In [48]:
def text_splitter(text):
    try:
        text_splitter = CharacterTextSplitter(
            chunk_size=7000,
            chunk_overlap=200,
            separator="\n",
        )
        texts = text_splitter.create_documents([text])
        return texts
    except Exception as e:
        print(f"Error al separar el texto: {e}")
        return []

### Modelo a utilizar (OllamaEmbeddings)

In [49]:
embedding = OllamaEmbeddings(
    model = "nomic-embed-text"
)

### BD vectorial (Chroma)

In [50]:
def get_vector_store(name_collection: str): 
    
    vector_store = Chroma(
    collection_name= name_collection,
    embedding_function=embedding,
    persist_directory="./vectorstore"
)    
    return vector_store

### Búsqueda de contenido relevante según el input ingresado 

In [51]:
def retrieval(input_user: str): 
    vector_store = get_vector_store("langchain")
    docs = vector_store.similarity_search(input_user)
    return docs

### Prompt para el modelo (PromptTemplate)

In [52]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre la energía solar y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "No cuento con información sobre eso".
    Utiliza siempre el contexto proporcionado para responder.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Conexión con el modelo y respuesta final (ChatGoogleGenerativeAI)

In [53]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.5-flash",
    temperature= 0.7
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

### Subir un PDF propio, aplicar las separaciones en el contenido y guardarlo en la BD vectorial

In [54]:
loader = upload_pdf("energia_solar.pdf")
texts = text_splitter(loader)
vector_store = get_vector_store("langchain")

vector_store.add_documents(texts)

['aa226b2d-8b84-4ef5-8fad-0930e55514f6']

### Prueba del RAG

In [55]:
input_user = input("Human: ")
print(input_user)

docs = retrieval(input_user=input_user)

print("\n============= Respuesta del modelo =============\n")
for chunk in response(input_user=input_user, contexto=docs):
    print(chunk, end=" ", flush=True)

¿Qué es la energía solar?

============= Respuesta del modelo =============

La energía solar es una fuente de energía renovable que se obtiene a partir de la radiación del sol.  